In [ ]:
# all necessary imports
import json
import csv
import pandas as pd

from transformers import pipeline
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import LongformerTokenizer, LongformerForSequenceClassification, EncoderDecoderModel, LongformerConfig, LongformerTokenizerFast
import numpy as np
import matplotlib.pyplot as plt
from textblob import TextBlob

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df= pd.read_csv('/Users/jyotit-kaushal/github/boozeless-analytics/data/singapore_processed_dataset.csv')

In [ ]:
df.info()

In [ ]:
# pivot table for venue_segment
pivot_table_venueseg = pd.pivot_table(df, 
                            index='venue_segment', 
                            aggfunc='size', 
                            fill_value=0)

print(pivot_table_venueseg)

In [ ]:
# sns.theme(palette= "#FF5B00")
segment_counts = df['venue_segment'].value_counts()

plt.figure(figsize=(8, 5))
barplot = sns.barplot(x=segment_counts.values, y=segment_counts.index, palette="Oranges", orient='horizontal')
plt.title('Number of Entries by Venue Segment', fontsize=14)
plt.xlabel('Count', fontsize=12)
plt.ylabel('Venue Segment', fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

for i, count in enumerate(segment_counts.values):
    barplot.text(count, i, f' {count}', va='center', fontsize=10)

plt.tight_layout()
plt.show()


In [ ]:
excluded_segments = ["Bar", "Entertainment", "Non Alcoholic", "Restaurant"]
df = df[df["venue_segment"].isin(excluded_segments)]

df.shape

In [ ]:
segment_counts = df['venue_segment'].value_counts()

plt.figure(figsize=(8, 5))
barplot = sns.barplot(x=segment_counts.values, y=segment_counts.index, palette="Oranges", orient='horizontal')
plt.title('Number of Entries by Venue Segment', fontsize=14)
plt.xlabel('Count', fontsize=12)
plt.ylabel('Venue Segment', fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

for i, count in enumerate(segment_counts.values):
    barplot.text(count, i, f' {count}', va='center', fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
sns.set_style("whitegrid")
df_temp= pd.read_json('/Users/jyotit-kaushal/github/boozeless-analytics/data/singapore_geo_dataset.json')
# Create a box plot
plt.figure(figsize=(8, 5))
sns.boxplot(x=df_temp['average_rating'], color='#FF5B00')
plt.title('Distribution of Average Ratings', fontsize=14)
plt.xlabel('Average Review Rating', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.tight_layout()
plt.show()

In [ ]:
sns.set_style("whitegrid")

# Create a box plot
plt.figure(figsize=(8, 5))
sns.boxplot(x=df['bayesian_weighted_rating'], color='#FF5B00')
plt.title('Distribution of Bayesian Weighted Ratings', fontsize=14)
plt.xlabel('Bayesian Weighted Rating', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.tight_layout()
plt.show()

In [ ]:
df = df[df['bayesian_weighted_rating'] >= 4]
df.shape

In [ ]:
pricepoints= df['price_point_bucket'].tolist()

def change_to_symbols(lst):
    symbols = {'$': 'Inexpensive', '$$': 'Moderate', '$$$': 'Expensive', '$$$$':'Very Expensive'}
    result = []

    for value in lst:
        if value in symbols:
            result.append(symbols[value])
        else:
            result.append(value)

    return result

df['price_point_bucket']= change_to_symbols(pricepoints)

pricepoint_counts = df['price_point_bucket'].value_counts()

plt.figure(figsize=(8, 5))
barplot = sns.barplot(x=pricepoint_counts.values, y=pricepoint_counts.index, palette="rocket", orient='horizontal')
plt.title('Number of Entries by Venue Segment', fontsize=14)
plt.xlabel('Count', fontsize=12)
plt.ylabel('Venue Segment', fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

for i, count in enumerate(pricepoint_counts.values):
    barplot.text(count, i, f' {count}', va='center', fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
# pivot table for price_point_bucket
pivot_table_pricepoint = pd.pivot_table(df, 
                            index='price_point_bucket', 
                            aggfunc='size', 
                            fill_value=0)

print(pivot_table_pricepoint)

In [ ]:
included_price_points= ['Expensive', 'Very Expensive']
df= df[df['price_point_bucket'].isin(included_price_points)]

df.shape

In [ ]:
df_boozeless= pd.read_csv('/Users/jyotit-kaushal/github/boozeless-analytics/data/restaurants_sorted_boozeless.csv')


In [ ]:
import folium

# Create a map centered around the mean latitude and longitude
map_center = [df_boozeless['latitude'].mean(), df_boozeless['longitude'].mean()]
mymap = folium.Map(location=map_center, zoom_start=12)

# Iterate over the DataFrame rows and add markers to the map
for index, row in df_boozeless.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['venue_name'],
        icon=folium.Icon(color='blue' if row['boozeless_rating'] == 'blue' else 'green' if row['boozeless_rating'] == 'green' else 'red')
    ).add_to(mymap)

# Display the map
mymap


In [ ]:
import folium
from folium.plugins import MarkerCluster
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable

# Function to convert boozeless_rating to color
def rating_to_color(rating):
    # Define colormap and normalization
    cmap = plt.cm.get_cmap('viridis')
    norm = Normalize(vmin=df_boozeless['boozeless_rating'].min(), vmax=df_boozeless['boozeless_rating'].max())
    # Map rating to color
    color = cmap(norm(rating))
    # Convert RGBA color to hex
    return plt.colors.rgb2hex(color)

# Create a map centered around the mean latitude and longitude
map_center = [df_boozeless['latitude'].mean(), df_boozeless['longitude'].mean()]
mymap = folium.Map(location=map_center, zoom_start=12)

# Create marker cluster for better performance with large datasets
marker_cluster = MarkerCluster().add_to(mymap)

# Iterate over the DataFrame rows and add markers to the map
for index, row in df_boozeless.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['venue_name'],
        icon=folium.Icon(color=rating_to_color(row['boozeless_rating']), icon='info-sign')
    ).add_to(marker_cluster)

# Display the map
mymap


In [ ]:
import folium
from folium.plugins import MarkerCluster
from matplotlib import colors
from matplotlib.cm import ScalarMappable

# Function to convert boozeless_rating to color
def rating_to_color(rating):
    # Define colormap and normalization
    cmap = plt.cm.get_cmap('viridis')
    norm = colors.Normalize(vmin=df_boozeless['boozeless_rating'].min(), vmax=df_boozeless['boozeless_rating'].max())
    # Map rating to color
    color = cmap(norm(rating))
    # Convert RGBA color to hex
    return colors.rgb2hex(color)

# Create a map centered around the mean latitude and longitude
map_center = [df_boozeless['latitude'].mean(), df_boozeless['longitude'].mean()]
mymap = folium.Map(location=map_center, zoom_start=12)

# Create marker cluster for better performance with large datasets
marker_cluster = MarkerCluster().add_to(mymap)

# Iterate over the DataFrame rows and add markers to the map
for index, row in df_boozeless.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['venue_name'],
        icon=folium.Icon(color=rating_to_color(row['boozeless_rating']), icon='info-sign')
    ).add_to(marker_cluster)

# Display the map
mymap


In [ ]:
import folium

# Function to convert boozeless_rating to color
def rating_to_color(rating):
    # Map rating to a color gradient
    color = (int(255 * (1 - rating)), int(255 * rating), 0)  # R: from 255 to 0, G: from 0 to 255, B: constant 0
    # Convert to hex format
    return '#{:02x}{:02x}{:02x}'.format(*color)

# Create a map centered around the mean latitude and longitude
map_center = [df_boozeless['latitude'].mean(), df_boozeless['longitude'].mean()]
mymap = folium.Map(location=map_center, zoom_start=12)

# Iterate over the DataFrame rows and add markers to the map
for index, row in df_boozeless.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['venue_name'],
        icon=folium.Icon(color=rating_to_color(row['boozeless_rating']), icon='info-sign')
    ).add_to(mymap)

# Display the map
mymap
